In [1]:
import pandas as pd

# parquet 파일 경로 지정
file_path = 'winograd_wsc.parquet'

# pyarrow(또는 fastparquet) 기반으로 parquet 파일 읽기
df = pd.read_parquet(file_path)

In [2]:
df['cleaning_status'].value_counts()

cleaning_status
verified     118
consensus     77
rejected       5
Name: count, dtype: int64

In [3]:
df.head()

,cleaning_status,platinum_prompt,platinum_prompt_no_cot,platinum_target,original_target,platinum_parsing_strategy,text,pronoun,pronoun_loc,quote,quote_loc,options,label,source
0,verified,Answer the question about the following phrase...,Answer the question about the following phrase...,[a],[a],multiple_choice,The city councilmen refused the demonstrators ...,they,63,they feared violence,63,"[The city councilmen, The demonstrators]",0,(Winograd 1972)
1,verified,Answer the question about the following phrase...,Answer the question about the following phrase...,[b],[b],multiple_choice,The city councilmen refused the demonstrators ...,they,63,they advocated violence,63,"[The city councilmen, The demonstrators]",1,(Winograd 1972)
2,verified,Answer the question about the following phrase...,Answer the question about the following phrase...,[a],[a],multiple_choice,The trophy doesn't fit into the brown suitcase...,it,55,it is too large,55,"[the trophy, the suitcase]",0,Hector Levesque
3,verified,Answer the question about the following phrase...,Answer the question about the following phrase...,[b],[b],multiple_choice,The trophy doesn't fit into the brown suitcase...,it,55,it is too small,55,"[the trophy, the suitcase]",1,Hector Levesque
4,verified,Answer the question about the following phrase...,Answer the question about the following phrase...,[a],[a],multiple_choice,Joan made sure to thank Susan for all the help...,she,47,she had received,47,"[Joan, Susan]",0,Hector Levesque


In [4]:
df['platinum_prompt'][0]

'Answer the question about the following phrase.\n\nPhrase: The city councilmen refused the demonstrators a permit because they feared violence.\n\nQuestion: What does "they" in "they feared violence" refer to in the phrase?\n\nOption A: The city councilmen\nOption B: The demonstrators\n\nThink step-by-step. Then, provide the final answer in the format "Answer: X" where X is the correct letter choice.'

In [4]:
import pandas as pd
import re

# 1. Read the question again: ...
def repeat_question(match):
    # match.group(1): 첫 번째 줄바꿈 (예: "\n\n" 또는 "\n   \n")
    # match.group(2): 질문 텍스트
    # match.group(3): 두 번째 줄바꿈 (예: "\n\n")
    # 원하는 결과: 첫 번째 줄바꿈 뒤 기존 질문 대신 "Read the question again:" + 질문 텍스트를 두 번 반복하고,
    # 두 번째 줄바꿈을 그대로 붙인다.
    return f"{match.group(1)}{match.group(2)}\n\nRead the question again: {match.group(2)}{match.group(3)}"

# \n\n 사이에 공백이 있을 수도 있으므로 \n\s*\n 형태로 패턴을 수정
df['RE2'] = df['platinum_prompt'].str.replace(
    r'(\n\s*\n)(.*?)(\n\s*\n)(?=Think step-by-step)',
    repeat_question,
    flags=re.DOTALL,
    regex=True
)

# # 2. Summarize the question first.
# df['sum'] = df['platinum_prompt'].str.replace(
#     r'(?=Think step-by-step)',
#     "Summarize the question first. ",
#     regex=True
# )

# # 3. Organize the question into a table first.
# df['table'] = df['platinum_prompt'].str.replace(
#     r'(?=Think step-by-step)',
#     "Organize the question into a table first. ",
#     regex=True
# )

# # 4. Organize the question into a graph structure first.
# df['graph'] = df['platinum_prompt'].str.replace(
#     r'(?=Think step-by-step)',
#     "Organize the question into a graph structure first. ",
#     regex=True
# )

# # 5. Summarize the question in bullet points first.
# df['bullet_point'] = df['platinum_prompt'].str.replace(
#     r'(?=Think step-by-step)',
#     "Summarize the question in bullet points first. ",
#     regex=True
# )

# 6. Read the question again first.
df['sRE2'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Read the question again first. ",
    regex=True
)

df['sRE2_m1'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Read the previous content again first. ",
    regex=True
)

df['sRE2_m2'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Read the phrase again first. ",
    regex=True
)


In [6]:
df['RE2'][3]

'Answer the question about the following phrase.\n\nPhrase: The trophy doesn\'t fit into the brown suitcase because it is too small.\n\nQuestion: What does "it" in "it is too small" refer to in the phrase?\n\nOption A: the trophy\nOption B: the suitcase\n\nRead the question again: Phrase: The trophy doesn\'t fit into the brown suitcase because it is too small.\n\nQuestion: What does "it" in "it is too small" refer to in the phrase?\n\nOption A: the trophy\nOption B: the suitcase\n\nThink step-by-step. Then, provide the final answer in the format "Answer: X" where X is the correct letter choice.'

In [8]:
df['platinum_prompt_no_cot'][0]

'Answer the question about the following phrase.\n\nPhrase: The city councilmen refused the demonstrators a permit because they feared violence.\n\nQuestion: What does "they" in "they feared violence" refer to in the phrase?\n\nOption A: The city councilmen\nOption B: The demonstrators\n\nThen, provide the final answer in the format "Answer: X" where X is the correct letter choice.'

In [7]:
import pandas as pd
import re

def repeat_question(match):
    # match.group(1): 첫 번째 줄바꿈 (예: "\n\n" 또는 "\n   \n")
    # match.group(2): 질문 텍스트
    # match.group(3): 두 번째 줄바꿈 (예: "\n\n")
    # 원하는 결과: 첫 번째 줄바꿈 뒤 기존 질문 대신 "Read the question again:" + 질문 텍스트를 두 번 반복하고,
    # 두 번째 줄바꿈을 그대로 붙인다.
    return f"{match.group(1)}{match.group(2)}\n\nRead the question again: {match.group(2)}{match.group(3)}"

# \n\n 사이에 공백이 있을 수도 있으므로 \n\s*\n 형태로 패턴을 수정
df['RE2_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(\n\s*\n)(.*?)(\n\s*\n)(?=Then, provide the final answer in the format)',
    repeat_question,
    flags=re.DOTALL,
    regex=True
)

# df['sum_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
#     r'(?=Then, provide the final answer in the format)',
#     "Summarize the question first. ",
#     regex=True
# )

# df['table_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
#     r'(?=Then, provide the final answer in the format)',
#     "Organize the question into a table first. ",
#     regex=True
# )

# df['graph_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
#     r'(?=Then, provide the final answer in the format)',
#     "Organize the question into a graph structure first. ",
#     regex=True
# )

# df['bullet_point_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
#     r'(?=Then, provide the final answer in the format)',
#     "Summarize the question in bullet points first. ",
#     regex=True
# )

df['sRE2_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer in the format)',
    "Read the question again first. ",
    regex=True
)

df['sRE2_m1_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer in the format)',
    "Read the previous content again first. ",
    regex=True
)

df['sRE2_m2_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer in the format)',
    "Read the phrase again first. ",
    regex=True
)

In [8]:
df['sRE2_no_cot'][0]

'Answer the question about the following phrase.\n\nPhrase: The city councilmen refused the demonstrators a permit because they feared violence.\n\nQuestion: What does "they" in "they feared violence" refer to in the phrase?\n\nOption A: The city councilmen\nOption B: The demonstrators\n\nRead the question again first. Then, provide the final answer in the format "Answer: X" where X is the correct letter choice.'

In [10]:
df.to_parquet('winograd_wsc_m.parquet')

In [11]:
df.head()

,cleaning_status,platinum_prompt,platinum_prompt_no_cot,platinum_target,original_target,platinum_parsing_strategy,text,pronoun,pronoun_loc,quote,...,label,source,RE2,sRE2,sRE2_m1,sRE2_m2,RE2_no_cot,sRE2_no_cot,sRE2_m1_no_cot,sRE2_m2_no_cot
0,verified,Answer the question about the following phrase...,Answer the question about the following phrase...,[a],[a],multiple_choice,The city councilmen refused the demonstrators ...,they,63,they feared violence,...,0,(Winograd 1972),Answer the question about the following phrase...,Answer the question about the following phrase...,Answer the question about the following phrase...,Answer the question about the following phrase...,Answer the question about the following phrase...,Answer the question about the following phrase...,Answer the question about the following phrase...,Answer the question about the following phrase...
1,verified,Answer the question about the following phrase...,Answer the question about the following phrase...,[b],[b],multiple_choice,The city councilmen refused the demonstrators ...,they,63,they advocated violence,...,1,(Winograd 1972),Answer the question about the following phrase...,Answer the question about the following phrase...,Answer the question about the following phrase...,Answer the question about the following phrase...,Answer the question about the following phrase...,Answer the question about the following phrase...,Answer the question about the following phrase...,Answer the question about the following phrase...
2,verified,Answer the question about the following phrase...,Answer the question about the following phrase...,[a],[a],multiple_choice,The trophy doesn't fit into the brown suitcase...,it,55,it is too large,...,0,Hector Levesque,Answer the question about the following phrase...,Answer the question about the following phrase...,Answer the question about the following phrase...,Answer the question about the following phrase...,Answer the question about the following phrase...,Answer the question about the following phrase...,Answer the question about the following phrase...,Answer the question about the following phrase...
3,verified,Answer the question about the following phrase...,Answer the question about the following phrase...,[b],[b],multiple_choice,The trophy doesn't fit into the brown suitcase...,it,55,it is too small,...,1,Hector Levesque,Answer the question about the following phrase...,Answer the question about the following phrase...,Answer the question about the following phrase...,Answer the question about the following phrase...,Answer the question about the following phrase...,Answer the question about the following phrase...,Answer the question about the following phrase...,Answer the question about the following phrase...
4,verified,Answer the question about the following phrase...,Answer the question about the following phrase...,[a],[a],multiple_choice,Joan made sure to thank Susan for all the help...,she,47,she had received,...,0,Hector Levesque,Answer the question about the following phrase...,Answer the question about the following phrase...,Answer the question about the following phrase...,Answer the question about the following phrase...,Answer the question about the following phrase...,Answer the question about the following phrase...,Answer the question about the following phrase...,Answer the question about the following phrase...
